In [0]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import copy
import os
from PIL import ImageFile
from PIL import Image


In [0]:
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [0]:
# Learning rate parameters
BASE_LR = 
EPOCH_DECAY =  
DECAY_WEIGHT =  
NUM_CLASSES = 
DATA_DIR = 
BATCH_SIZE = 
NUM_EPOCHS = 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)
GPU_MODE = 1 # set to 1 if want to run on gpu.




In [0]:
# If you want to read more, transforms is a function from torchvision, and you can go read more here - http://pytorch.org/docs/master/torchvision/transforms.html
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        #transforms.ColorJitter(brightness=0, contrast=0.5, saturation=0.5, hue=0),
        #transforms.RandomRotation(degrees=(180,-180)),
        #transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3, fill=0),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    

    
}

In [0]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train','val','test']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=0
                                               )
                for x in [ 'train','val','test']}


In [0]:
.
num_epochs = NUM_EPOCHS

#RESNET
# model = models.resnet50(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
# model.to(device)


#VGG-16
# model = models.vgg16_bn(pretrained=True)
# num_features = model.classifier[6].in_features
# features = list(model.classifier.children())[:-1] 
# features.extend([nn.Linear(num_features, NUM_CLASSES)]) 
# model.classifier = nn.Sequential(*features)
# model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

#optimizer = optim.RMSprop(model.parameters(), lr=0.0001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=DECAY_WEIGHT)

# Train the model
lr = 0.0001
total_step = len(dset_loaders['train'])
loss_train = []
loss_val = []
best_accuracy = None
accuracy_val = []
best_model = model
for epoch in range(num_epochs):
    print('Epoch:', epoch+1)
    
    model.train()

    loss_iter = 0
    for i, (images, labels) in enumerate(dset_loaders['train']):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_iter += loss.item()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    loss_train.append(loss_iter/(len(dset_loaders['train'])*BATCH_SIZE))


    # Code to update the lr
    lr *= 0.99
    update_lr(optimizer, lr)
    
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        loss_iter = 0
        for images, labels in dset_loaders['val']:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss = criterion(outputs, labels)
            loss_iter += loss.item()
            
        loss_val.append(loss_iter/(len(dset_loaders['val'])*BATCH_SIZE))

        accuracy = 100 * correct / total
        accuracy_val.append(accuracy)
        
        print('Validataion accuracy is: {} %'.format(accuracy))
        

        if epoch >=3 and accuracy > max(accuracy_val[:-1]):
            best_model = copy.deepcopy(model)
            early_epoch = epoch
        
        if epoch == num_epochs - 1:
            print(f'\nEarly Stopping at the epoch: {early_epoch+1} with accuracy: {accuracy_val[early_epoch]:.2f} and acc: {100 * accuracy_val[early_epoch]:.2f}%')


In [0]:
correct = 0
total = 0
for images, labels in dset_loaders['test']:
  images = images.to(device)
  labels = labels.to(device)
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))